# 1.4 The linear programming model in AMPL

In [1]:
# install dependencies
%pip install -q amplpy numpy pandas
from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Now we can talk about AMPL.  The AMPL language is intentionally as
close to the mathematical form as it can get while still being easy
to type on an ordinary keyboard and to process by a program.  There are
AMPL constructions for each of the basic components listed above 
&ndash;
sets, parameters, variables, objectives, and constraints 
&ndash;
and ways to write arithmetic expressions, sums over sets, and so on.

We first give an AMPL model that resembles our algebraic model as
much as possible, and then present an improved version that takes
better advantage of the language.

## The basic model

For the basic production model of [Section 1.1](tut_1_1.ipynb#fig-1-1), a direct
transcription into AMPL would look like [Figure 1-2](#fig-1-2).

<a id='fig-1-2'><center><b>Figure 1-2:</b> Basic production model in AMPL (file prod.mod).</center></a>

In [2]:
%%writefile prod.mod

set P;

param a {j in P};
param b;
param c {j in P};
param u {j in P};

var X {j in P};

maximize Total_Profit: sum {j in P} c[j] * X[j];

subject to Time: sum {j in P} (1/a[j]) * X[j] <= b;

subject to Limit {j in P}: 0 <= X[j] <= u[j];

Overwriting prod.mod


The keyword
*set*
declares a set name, as in

```
set P;
```

The members of set
*P*
will be provided in separate data statements, which we'll show in a
moment.

The keyword
*param*
declares a parameter, which may be a single scalar value, as in
```
param b;
```
or a collection of values indexed by a set.  Where algebraic notation
says that "there is an $a_j$ for each $j$ in $P$", one writes in
AMPL
```
param a {j in P};
```
which means that
*a*
is a collection of parameter values, one for each member of the set
*P* .
Subscripts in algebraic notation are written with square brackets in AMPL,
so an individual value like $a_j$ is written
`a[j]` .

The
*var*
declaration
```
var X {j in P};
```
names a collection of variables, one for each member of
*P* ,
whose values the solver is to determine.

The objective is given by the declaration
```
maximize Total_Profit: sum {j in P} c[j] * X[j];
```
The name
*Total_Profit*
is arbitrary; a name is required by the syntax, but any name will do.
The precedence of the
*sum*
operator is lower than that of
*\** ,
so the expression is indeed a sum of products, as intended.

Finally, the constraints are given by
```
subject to Time: sum {j in P} (1/a[j]) * X[j] <= b;
subject to Limit {j in P}: 0 <= X[j] <= u[j];
```

The
*Time*
constraint says that a certain sum over the set
*P*
may not exceed the value of parameter
*b* .
The
*Limit*
constraint is actually a family of constraints, one for each member
*j*
of
*P* :
each
`X[j]`
is bounded by zero and the corresponding
`u[j]` .

The construct
`{j in P}`
is called an
*indexing expression*.
As you can see from our example,
indexing expressions are used not only in declaring parameters and
variables, but in any context where the algebraic model does
something "for each *j* in *P*".  Thus the
`Limit`
constraints are declared
```
subject to Limit {j in P}
```
because we want to impose a different restriction
`0 <= X[j] <= u[j]`
for each different product $j$ in the set $P$.  In the same way, the
summation in the objective is written
```
sum {j in P} c[j] * X[j]
```
to indicate that the different terms
`c[j] * X[j]` ,
for each
`j`
in the set
`P` ,
are to be added together in computing
the profit.

The layout of an AMPL model is quite free. Sets, parameters, and
variables must be declared before they are used but can otherwise
appear in any order.  Statements end with semicolons and can be
spaced and split across lines to enhance readability.
Upper and
lower case letters are different, so
`time` ,
`Time` ,
and
`TIME`
are three different names.

You have undoubtedly noticed several places where traditional
mathematical notation has been adapted in AMPL to the limitations of
normal keyboards and character sets.  AMPL uses
the word
`sum`
instead of $\sum$ to express a summation, and
`in`
rather than $\in$ for set membership.  Set specifications are
enclosed in braces, as in
`{j in P}` .
Where mathematical notation uses adjacency to signify
multiplication in $c_j X_j$, AMPL uses the
`*`
operator of most programming languages, and subscripts are
denoted by brackets, so $c_j X_j$ becomes
`c[j]*X[j]` .

You will find that the rest of AMPL is similar - a few more
arithmetic operators, a few more key words like
`sum`
and
`in` ,
and many more ways to specify indexing expressions.  Like any other
computer language, AMPL has a precise grammar, but we won't stress
the rules too much here; most will become clear as we go along, and
full details are given in the reference manual, Appendix @Refman@.

Our original two-variable linear program is one of the
many LPs that are instances of the [Figure 1-2](#fig-1-2) model.
To specify it or any
other such instance, we need to supply the membership of
`P`
and the values of the various parameters.  There is no standard way
to describe these data values in algebraic notation; usually some
kind of informal tables are used, such as the ones we showed earlier.
The Python programming language provides different native data structures
that are quite convenient to load the data into optimization problems.
Figure [Figure 1-3](#fig-1-3) gives data for the basic production model
in that form.

A
`list`
supplies the members
*bands*
and
*coils*
of set
`P` ,
and a Pandas Dataframe
gives the corresponding values for
*a* ,
*c* ,
and
*u* .
A simple
assignment gives the value for
*b* .
These and other data statements are described in detail in
Appendix xTODO, have a variety of options that let you list or
tabulate parameters in convenient ways.

<a id='fig-1-3'><center><b>Figure 1-3:</b> Production model data.</center></a>

In [3]:
import pandas as pd
import numpy as np

def load_data(ampl):
    P = ['bands', 'coils']
    # Tabla for a, c, and u
    P_df = pd.DataFrame(
        np.array(
            [
                [200, 25, 6000],
                [140, 30, 4000],
            ]
        ),
        columns=['a','c','u'],
        index=P,
    )

    ampl.set['P'] = P
    ampl.set_data(P_df, "P")
    ampl.param['b'] = 40

## An improved model

We could go on immediately to solve the linear program defined by
Figures
[1-2](#fig-1-2)
and
[1-3](#fig-1-3)
. Once we have
written the model in AMPL, however, we need not feel constrained by
all the conventions of algebra, and we can instead consider changes
that might make the model easier to work with.  Figures
[Figure 1-4a](#fig-1-4a)
and
[Figure 1-4b](#fig-1-4b)
show a possible
"improved" version.  The short "mathematical" names for the sets,
parameters and variables have been replaced by longer, more
meaningful ones.  The indexing expressions have become
`{p in PROD}` ,
or just
`{PROD}`
in those declarations that do not use the index
`p` .
The bounds on variables have been placed within their
`var`
declaration, rather than in a separate constraint; analogous bounds
have been placed on the parameters, to indicate the ones that must
be positive or nonnegative in any meaningful linear program derived
from the model.

<a id='fig-1-4a'><center><b>Figure 1-4a:</b> Steel production model (steel.mod).</center></a>

In [4]:
%%writefile steel.mod

set PROD;  # products

param rate {PROD} > 0;     # tons produced per hour
param avail >= 0;          # hours available in week

param profit {PROD};       # profit per ton
param market {PROD} >= 0;  # limit on tons sold in week

var Make {p in PROD} >= 0, <= market[p]; # tons produced

# Objective: total profits from all products
maximize Total_Profit: sum {p in PROD} profit[p] * Make[p];

# Constraint: total of hours used by all
# products may not exceed hours available
subject to Time: sum {p in PROD} (1/rate[p]) * Make[p] <= avail;         

Overwriting steel.mod


<a id='fig-1-4b'><center><b>Figure 1-4b:</b> Data for steel production model.</center></a>

In [5]:
def load_data(ampl):
    PROD = ['bands', 'coils']
    # Tabla for a, c, and u
    PROD_df = pd.DataFrame(
        np.array(
            [
                [200, 25, 6000],
                [140, 30, 4000],
            ]
        ),
        columns=['rate','profit','market'],
        index=PROD,
    )

    ampl.set['PROD'] = PROD
    ampl.set_data(PROD_df, "PROD")
    ampl.param['avail'] = 40

Finally, comments have been added to help explain the model to a
reader.  Comments begin with
`#`
and end at the end of the line.
As in any programming language,
judicious use of meaningful names, comments and formatting helps
to make AMPL models more readable and understandable.

There are always many ways to describe a particular model in AMPL.
It is left to the modeler to pick the way that seems clearest or most
convenient.  Our earlier, mathematical approach is often preferred
for working quickly with a familiar model.  On the other hand, the
second version is more attractive for a model that will be
maintained and modified by several people over months or years.

If we put all of the model
declarations into a file called
`steel.mod` ,
and call the `load_data()` function ,
then as before a solution can be found and displayed by typing just a few statements:

In [6]:
ampl = AMPL()
ampl.read('steel.mod')
load_data(ampl)
ampl.solve(solver='highs')
ampl.display('Make')

HiGHS 1.8.1: optimal solution; objective 192000
1 simplex iterations
0 barrier iterations
Make [*] :=
bands  6000
coils  1400
;



The
`read()`
function specifies a file to be read, in this case the model from
`steel.mod` ,
which encourages a clean separation of model
from data loaded via Python statements.

Filenames can have any form recognized by your computer's
operating system; AMPL doesn't check them for correctness.
The filenames here and in the rest of the book
refer to example files that are available from the AMPL web site
and other AMPL distributions.

Once the model has been solved, we can show the optimal
values of all of the variables
`Make[p]` ,
by typing
`ampl.display('Make')` .
(The
`[*]`
indicates a variable or parameter with a single subscript.)

As an illustrative example, conveying the model coefficients and the solution values as dictionaries in Python.

In [7]:
# data defined in Python data structures
PROD = ['bands', 'coils']
rate = {'bands' : 200, 'coils' : 140}
profit = {'bands' : 25, 'coils' : 30}
market = {'bands' : 6000, 'coils' : 4000}
avail = 40

# instantiate AMPL object
ampl = AMPL()
# read the model
ampl.read('steel.mod')
# read the data
ampl.set['PROD'] = PROD
ampl.param['rate'] = rate
ampl.param['profit'] = profit
ampl.param['market'] = market
ampl.param['avail'] = avail
# choose solver and solve
ampl.solve(solver='highs')

# display results for the 'Make' variable
ampl.display('Make')

# get the 'Make' variable results in Python and print the values
make = ampl.var['Make'].to_dict()
print('make:', make)

HiGHS 1.8.1: optimal solution; objective 192000
1 simplex iterations
0 barrier iterations
Make [*] :=
bands  6000
coils  1400
;

make: {'bands': 6000, 'coils': 1400}


## Catching errors

You will inevitably make some mistakes as you develop a model.  AMPL
detects various kinds of incorrect statements, which
are reported in error messages following the
`read()` ,
or
`solve()`
commands, in addition to data assignments.

AMPL catches many errors as soon as the model is read.
For example, if you use the wrong syntax for the
bounds in the declaration of the variable
`Make` ,
you will receive an error message like this, right after you enter the
`model`
command:

```
steel.mod, line 8 (offset 250):
        syntax error
context: var Make {p in PROD} >>> 0 <<< <= Make[p] <= market[p];
```

If you inadvertently use
`make`
instead of
`Make`
in an expression like
`profit[p] * make[p]` ,
you will receive this message:

```
steel.mod, line 11 (offset 339):
        make is not defined
context: maximize Total_Profit:
              sum {p in PROD} profit[p] *  >>> make[p] <<< ;
```

In each case, the offending line is printed, with the approximate
location of the error surrounded by
`>>>`
and
`<<<` .

Other common sources of error messages include a model component used before it is declared,
a missing semicolon at the end of a command, or a reserved word like
`sum`
or
`in`
used in the wrong context.
(Section xTODO contains a list of reserved words.)

Errors in the data values are caught after you type
`solve` .
If the number of hours were given as -40, for instance, you would
see:

```
> ampl.solve(solver='highs')
Error executing "solve" command:
error processing param avail:
        failed check: param avail = -40
                is not >= 0;
```

It is good practice to include as many validity checks as possible in
the model, so that errors are caught at an early stage.

Despite your best efforts to formulate the model correctly and to include validity
checks on the data, sometimes a model that generates no error messages and that
elicits an "optimal solution" report from the solver will nonetheless produce a
clearly wrong or meaningless solution.  All of the production levels might be zero, for example,
or the product with a lower profit per hour may be produced at a higher volume.  In
cases like these, you may have to spend some time reviewing your formulation before
you discover what is wrong.

The
`expand`
command can be helpful in your search for errors, by showing you how AMPL instantiated
your symbolic model.  To see what AMPL generated for the objective
`Total_Profit` ,
for example, you could type:

xTODO needs an API update

In [8]:
#ampl.eval("expand Total_Profit;")
ampl.expand('Total_Profit')

AttributeError: 'amplpy.ampl.AMPL' object has no attribute 'expand'

This corresponds directly to our explicit formulation back in [Section 1.1](./tut_1_1.ipynb).  Expanding
the constraint works similarly:

In [ ]:
#ampl.eval("expand Time;")
ampl.expand('Time')

Expressions in the symbolic model, such as the coefficients
`1/rate[p]`
in this example, are evaluated before the expansion is displayed.  You can expand
the objective and all of the constraints at once by typing
`expand`
by itself.

The expressions above show that the symbolic model's
`Make[j]`
expands to the explicit variables
`Make['bands']`
and
`Make['coils']` .
You can use expressions like these in AMPL commands, for example to expand a
particular variable to see what coefficients it has in the objective and
constraints:

In [ ]:
ampl.expand("Make['coils']")
#ampl.eval("expand Make['coils'];")

Either single quotes
(')
or double quotes
(")
may surround the subscript.